<a href="https://colab.research.google.com/github/frmfelipemaia/Global_Solution_IA/blob/main/GS_Green_Energy_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Solution
## Tema: Green Energy
## Disciplina:  Inteligência Artificial e Computacional
## Turmas: 3ECA e 3ECR (2o semestre de 2024)
## Professor: André Tritiack

Desenvolvimento de um sistema de Inteligência Artificial que utilize técnicas de Machine Learning e/ou Deep
Learning visando a otimização do gerenciamento do carregamento de veículos elétricos, considerando
aspectos como previsão de demanda, gestão de recursos energéticos e redução de custos. Utilize técnicas
de Machine Learning com dados tabulares, sistemas de Visão Computacional e/ou integração com LLMs /
IAs generativas.

**Integrantes**


* FELIPE RODRIGUES MAIA 	             RM 94234
* MATEUS DE ALMEIDA MARTINS BOIHAGIAN  RM 93434
* ITHAN CASSIANO ALBUQUERQUE PEREIRA   RM 94931


---




## Parte 1: Importação de bibliotecas

In [29]:
import pandas as pd
import numpy as np

# Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# Treino e teste
from sklearn.model_selection import train_test_split

# Usaremos o KMeans de uma maneira diferente, ele agrupará a latitue e longitude em regiões.
from sklearn.cluster import KMeans

# Conversão de dados categóricos em numéricos:
from sklearn.preprocessing import LabelEncoder

# Modelos que serão utilizados no projeto.
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# Importando as bibliotecas necessárias para avaliação de modelos
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Parte 2: Importação dos dados e Análise Exploratória

In [2]:
dados = pd.read_csv('dados.csv')

# Eliminamos a primeira coluna pois o nome da estação não é relevante pro modelo.
dados = dados.drop(['Station Name'], axis=1)

# Assim como a segunda coluna pois já temos as coordenadas para agrupamento regional.
dados = dados.drop(['Street Address'], axis=1)

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column
0,Darien,24 hours daily,NONE,2,NONE,NONE,POINT (-73.4764687 41.072882)
1,Meriden,24 hours daily; for Tesla use only,NONE,NONE,8,NONE,POINT (-72.773473 41.527367)
2,Beacon Falls,24 hours daily,NONE,1,NONE,NONE,POINT (-73.065583 41.44548100000001)
3,Old Saybrook,24 hours daily,NONE,2,NONE,NONE,POINT (-72.3825 41.3102778)
4,Fairfield,24 hours daily,NONE,2,NONE,NONE,POINT (-73.264511 41.143125)


In [3]:
# Exibir informações gerais do dataset
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   City                      385 non-null    object
 1   Access Days Time          385 non-null    object
 2   EV Level1 EVSE Num        385 non-null    object
 3   EV Level2 EVSE Num        385 non-null    object
 4   EV DC Fast Count          385 non-null    object
 5   EV Other Info             385 non-null    object
 6   New Georeferenced Column  385 non-null    object
dtypes: object(7)
memory usage: 21.2+ KB


In [4]:
# Estatísticas descritivas das colunas numéricas
dados.describe()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column
count,385,385,385,385,385,385,385
unique,119,53,5,14,10,3,384
top,Stamford,24 hours daily,NONE,2,NONE,NONE,POINT (-72.489406 41.983904)
freq,21,222,376,179,331,383,2


*Podemos observar que existem 119 cidades únicas, isso nos ajudará com um dos problemas propostos. As cidades únicas podem ser usadas para prever a demanda regional*

In [5]:
# Verificar valores ausentes em cada coluna
valores_vazios = dados.isnull().sum()
print(valores_vazios)

City                        0
Access Days Time            0
EV Level1 EVSE Num          0
EV Level2 EVSE Num          0
EV DC Fast Count            0
EV Other Info               0
New Georeferenced Column    0
dtype: int64


**Nenhuma coluna incompleta.**

In [6]:
# Contagem de valores únicos em cada coluna
# Redundancia nesses dados apenas para deixar separado para futura analise, os mesmo estão descritos acima
# no describe().
valores_unicos = dados.nunique()
print(valores_unicos)

City                        119
Access Days Time             53
EV Level1 EVSE Num            5
EV Level2 EVSE Num           14
EV DC Fast Count             10
EV Other Info                 3
New Georeferenced Column    384
dtype: int64


## Pré Processamento dos dados

In [7]:
# 1. Separar latitude e longitude para agrupamento regional para resolver um dos problemas propostos.
dados[['Latitude', 'Longitude']] = dados['New Georeferenced Column'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)').astype(float)

# E agora não precisamos mais da coluna New Georeferenced Column
dados = dados.drop(['New Georeferenced Column'], axis=1)

# E por fim, vou separar a latitude e longitude em regiões, para melhorar a coorelação dos dados de localização e uso diário.

# Criar clusters para identificar regiões
kmeans = KMeans(n_clusters=5, random_state=42)
dados['Region'] = kmeans.fit_predict(dados[['Latitude', 'Longitude']])
dados = dados.drop(columns=['Latitude', 'Longitude'])

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region
0,Darien,24 hours daily,NONE,2,NONE,NONE,2
1,Meriden,24 hours daily; for Tesla use only,NONE,NONE,8,NONE,1
2,Beacon Falls,24 hours daily,NONE,1,NONE,NONE,1
3,Old Saybrook,24 hours daily,NONE,2,NONE,NONE,3
4,Fairfield,24 hours daily,NONE,2,NONE,NONE,2


In [8]:
# 2. Converter colunas de números para valores numéricos para remover os NONE que afetariam o modelo.
for col in ['EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count']:
    dados[col] = pd.to_numeric(dados[col].replace('NONE', 0), errors='coerce')
dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region
0,Darien,24 hours daily,0,2,0,NONE,2
1,Meriden,24 hours daily; for Tesla use only,0,0,8,NONE,1
2,Beacon Falls,24 hours daily,0,1,0,NONE,1
3,Old Saybrook,24 hours daily,0,2,0,NONE,3
4,Fairfield,24 hours daily,0,2,0,NONE,2


In [9]:
# 3. Codificar variáveis categóricas para que o modelo possa ser mais facilmente ensinado.
dados['City'] = dados['City'].astype('category').cat.codes
dados['Access Days Time'] = dados['Access Days Time'].astype('category').cat.codes
dados['EV Other Info'] = dados['EV Other Info'].astype('category').cat.codes

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region
0,17,2,0,2,0,2,2
1,48,5,0,0,8,2,1
2,1,2,0,1,0,2,1
3,72,2,0,2,0,2,3
4,28,2,0,2,0,2,2


## Agora adicionaremos váriaveis ao dataset que serão os 3 problemas que estamos interessados em que o modelo resolva, sendo eles

* Previsão de Demanda
* Gestão de Recursos Energéticos
* Redução de Custos Operacionais




In [10]:
# Para os valores inicias da previsão de demanda, utilizamos essas variáveis porque refletem diretamente
# a capacidade operacional da estação, que influencia diretamente na demanda prevista. A multiplicação foi
# realizada para atribuir diferentes pesos para cada carregador, sendo o Level1 um carregador mais lento
# que vai afetar a demanda sendo menos utilizados por usuários que precisam de carregamento rápido, e assim
# em diante para os demais.

dados['Previsao_Demanda'] = (
    dados['EV Level1 EVSE Num'] * 0.5 +
    dados['EV Level2 EVSE Num'] * 0.8 +
    dados['EV DC Fast Count'] * 1.2
)

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region,Previsao_Demanda
0,17,2,0,2,0,2,2,1.6
1,48,5,0,0,8,2,1,9.6
2,1,2,0,1,0,2,1,0.8
3,72,2,0,2,0,2,3,1.6
4,28,2,0,2,0,2,2,1.6


In [11]:
# Para a gestão de recursos, as mesmas váriaveis foram escolhidas, dessa vez apenas sua quantidade
# númerica, porque definem os limites físicos e energéticos de cada estação,
# permitindo planejar e alocar energia de forma eficiente

dados['Gestao_Recursos'] = (
    dados['EV Level1 EVSE Num'] +
    dados['EV Level2 EVSE Num'] +
    dados['EV DC Fast Count']
)

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region,Previsao_Demanda,Gestao_Recursos
0,17,2,0,2,0,2,2,1.6,2
1,48,5,0,0,8,2,1,9.6,8
2,1,2,0,1,0,2,1,0.8,1
3,72,2,0,2,0,2,3,1.6,2
4,28,2,0,2,0,2,2,1.6,2


In [12]:
# Por fim, a varíavel de Redução de Custos que vai começar sendo inversamente proporcional a previsão
# de demanda, pois durante o horário de pico, muitos veículos utilizam a estação, exigindo mais energia e manutenção, aumentando os custos.
# e durante a noite ou em locais menos movimentados, há menor consumo e custos associados. Logo o custo cai.

dados['Reducao_Custos'] = 1 / (dados['Previsao_Demanda'] + 1e-5)

dados.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region,Previsao_Demanda,Gestao_Recursos,Reducao_Custos
0,17,2,0,2,0,2,2,1.6,2,0.624996
1,48,5,0,0,8,2,1,9.6,8,0.104167
2,1,2,0,1,0,2,1,0.8,1,1.249984
3,72,2,0,2,0,2,3,1.6,2,0.624996
4,28,2,0,2,0,2,2,1.6,2,0.624996


## Agora iremos separar os dados de treino e os dados de teste, serão criados 3 modelos pois cada variável-alvo mede um aspecto distinto do problema, com relações específicas com os dados de entrada.

Para a Previsão de Demanda, usaremos Random Forest Regressor.

Para a Gestão de Recursos Energéticos, usaremos a Regressão Linear.

Para a Redução de Custos Operacionais, usaremos um modelo que não conhecíamos mas pelas nossas pesquisas, parecia se adequar melhor ao projeto, Gradient Boosting Regressor.

In [13]:
X = dados.drop(columns=['Previsao_Demanda', 'Gestao_Recursos', 'Reducao_Custos'])
X.head()

,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,Region
0,17,2,0,2,0,2,2
1,48,5,0,0,8,2,1
2,1,2,0,1,0,2,1
3,72,2,0,2,0,2,3
4,28,2,0,2,0,2,2


In [14]:
y_demanda = dados['Previsao_Demanda']

In [15]:
y_recursos = dados['Gestao_Recursos']

In [16]:
y_custos = dados['Reducao_Custos']



---



**Separação de treino e teste para cada modelo.**

In [23]:
X_train_demanda, X_test_demanda, y_train_demanda, y_test_demanda = train_test_split(
    X, y_demanda, test_size=0.3, random_state=70
)

In [24]:
X_train_recurso, X_test_recurso, y_train_recurso, y_test_recurso = train_test_split(
    X, y_recursos, test_size=0.3, random_state=15
)

In [36]:
X_train_custos, X_test_custos, y_train_custos, y_test_custos = train_test_split(
    X, y_custos, test_size=0.3, random_state=9
)

## Modelo 1: Previsão de Demanda (Random Forest)

In [26]:
model_demanda = RandomForestRegressor(random_state=70)

In [27]:
model_demanda.fit(X_train_demanda, y_train_demanda)

RandomForestRegressor(random_state=70)

In [28]:
y_pred_demanda = model_demanda.predict(X_test_demanda)

**Avaliação do primeiro modelo, para previsão de demanda.**

In [30]:
mae_demanda = mean_absolute_error(y_test_demanda, y_pred_demanda)
mse_demanda = mean_squared_error(y_test_demanda, y_pred_demanda)
r2_demanda = r2_score(y_test_demanda, y_pred_demanda)

## Modelo 2: Gestão de Recursos (Regressão Linear)

In [31]:
model_recursos = LinearRegression()

In [32]:
model_recursos.fit(X_train_recurso, y_train_recurso)

LinearRegression()

In [33]:
y_pred_recursos = model_recursos.predict(X_test_recurso)

**Avaliação do segundo modelo, para Gestão de Recursos.**

In [34]:
mae_recursos = mean_absolute_error(y_test_recurso, y_pred_recursos)
mse_recursos = mean_squared_error(y_test_recurso, y_pred_recursos)
r2_recursos = r2_score(y_test_recurso, y_pred_recursos)

## Modelo 3: Redução de Custos (Gradient Boosting)

In [37]:
model_custos = GradientBoostingRegressor(random_state=9)

In [38]:
model_custos.fit(X_train_custos, y_train_custos)

GradientBoostingRegressor(random_state=9)

In [39]:
y_pred_custos = model_custos.predict(X_test_custos)

**Avaliação do terceiro modelo, para Redução de Custos.**

In [40]:
mae_custos = mean_absolute_error(y_test_custos, y_pred_custos)
mse_custos = mean_squared_error(y_test_custos, y_pred_custos)
r2_custos = r2_score(y_test_custos, y_pred_custos)

# Conclusão, análise do resultado dos 3 modelos.

In [42]:
print("Avaliação do Modelo 1 (Previsão de Demanda):")
print("MAE:", mae_demanda)
print("MSE:", mse_demanda)
print("R²:", r2_demanda)

Avaliação do Modelo 1 (Previsão de Demanda):
MAE: 0.04446551724138093
MSE: 0.012414155172413411
R²: 0.9974182217779254


In [44]:
print("Avaliação do Modelo 2 (Gestão de Recursos):")
print("MAE:", mae_recursos)
print("MSE:", mse_recursos)
print("R²:", r2_recursos)

Avaliação do Modelo 2 (Gestão de Recursos):
MAE: 4.938578281953282e-16
MSE: 1.4085587499474317e-30
R²: 1.0


In [45]:
print("Avaliação do Modelo 3 (Redução de Custos):")
print("MAE:", mae_custos)
print("MSE:", mse_custos)
print("R²:", r2_custos)

Avaliação do Modelo 3 (Redução de Custos):
MAE: 0.0035380431949713805
MSE: 8.11173142941037e-05
R²: 0.9993367170546467


## **O que podemos concluir com os 3 modelos?**

Acreditamos que nossos modelos usufruiram muito bem do dataset fornecido pelo professor, podendo prever de maneira confiável os 3 problemas propostos, com esses modelos, podemos levar para um caso real e poder tomar decisões com segurança.

Todos os modelos têm desempenho excepcional, com altos valores de R² (> 0.99) e erros muito baixos.

Os modelos estão prontos para implementação em um sistema real de gerenciamento de estações de carregamento.


# Modelo Bônus, Rede Neural

Após o sucesso de nossos modelos, decidimos expandir o projeto com uma rede neural ensinada pelo professor, para analizar o seu desempenho contra os modelos tradicionais.

In [60]:
# Importação do Keras assim como de camadas densas e modelos de otimização.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [49]:
# Separação de dados de entrada e de saída.

X = dados.drop(columns=['Previsao_Demanda', 'Gestao_Recursos', 'Reducao_Custos'])
y = dados[['Previsao_Demanda', 'Gestao_Recursos', 'Reducao_Custos']]

In [59]:
# Criação das amostras de teste e treino para o modelo Keras.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

## Criação do modelo.

In [61]:
modeloRNA = Sequential()

In [62]:
# Camada de entrada e primeira camada oculta
modeloRNA.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# Segunda camada oculta
modeloRNA.add(Dense(units=32, activation='relu'))

In [64]:
# Camada de saída com 3 neuronios.
modeloRNA.add(Dense(units=3, activation='linear'))

In [66]:
# Compilando o modelo xD
modeloRNA.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [67]:
# Exibir o sumário do modelo
modeloRNA.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 128)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,251 (20.51 KB)

 Trainable params: 5,251 (20.51 KB)

 Non-trainable params: 0 (0.00 B)

Agora com o modelo criado, podemos treina-lo :)

In [68]:
history = modeloRNA.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 19.3338 - mae: 3.3872 - val_loss: 5.5766 - val_mae: 1.6396
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 5.8700 - mae: 1.5799 - val_loss: 3.7257 - val_mae: 1.2311
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.0439 - mae: 1.2247 - val_loss: 3.2722 - val_mae: 1.0585
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.0030 - mae: 1.0937 - val_loss: 3.0806 - val_mae: 1.1095
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3773 - mae: 1.1146 - val_loss: 2.6888 - val_mae: 0.9233
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.0637 - mae: 0.9815 - val_loss: 2.4934 - val_mae: 0.9511
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7500 - mae: 0.9604 - val_loss: 2.2367 - val_mae: 0.8642
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.7922 - mae: 0.9169 - val_loss: 1.9987 - val_mae: 0.8366
Epoch 9/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.26

## Avaliações do modelo Keras

In [71]:
evaluation = modeloRNA.evaluate(X_test, y_test)
print(f"(MSE): {evaluation[0]},(MAE): {evaluation[1]}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0313 - mae: 0.1298 
(MSE): 0.03539074957370758,(MAE): 0.1324291229248047


In [72]:
# Fazer previsões
y_pred = modeloRNA.predict(X_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [76]:
# Separar as previsões para cada variável
y_pred_demanda = y_pred[:, 0]
y_pred_recursos = y_pred[:, 1]
y_pred_custos = y_pred[:, 2]

In [81]:
# Avaliação separada para cada variável
print("Avaliação do Modelo:")
print("Previsão de Demanda:")
print(f"  MAE: {mean_absolute_error(y_test_np[:, 0], y_pred_demanda)}")
print(f"  MSE: {mean_squared_error(y_test_np[:, 0], y_pred_demanda)}")
print(f"  R²: {r2_score(y_test_np[:, 0], y_pred_demanda)}\n")

print("Gestão de Recursos:")
print(f"  MAE: {mean_absolute_error(y_test_np[:, 1], y_pred_recursos)}")
print(f"  MSE: {mean_squared_error(y_test_np[:, 1], y_pred_recursos)}")
print(f"  R²: {r2_score(y_test_np[:, 1], y_pred_recursos)}\n")

print("Redução de Custos:")
print(f"  MAE: {mean_absolute_error(y_test_np[:, 2], y_pred_custos)}")
print(f"  MSE: {mean_squared_error(y_test_np[:, 2], y_pred_custos)}")
print(f"  R²: {r2_score(y_test_np[:, 2], y_pred_custos)}")

Avaliação do Modelo:
Previsão de Demanda:
  MAE: 0.09871412020225034
  MSE: 0.017526684492260743
  R²: 0.9961446787413365

Gestão de Recursos:
  MAE: 0.116095938465812
  MSE: 0.04446792476265159
  R²: 0.9898039165473833

Redução de Custos:
  MAE: 0.18247721092180946
  MSE: 0.04417761935995117
  R²: 0.609797264606698


## Resultados interessantes, a redução de custo teve um R quadrado mais baixo que o modelo tradicional, entretanto as outras previões estão ótimas, o modelo Keras se mostrou muito promissor para resolver os três problemas que foram passados pelo professor.